# Package Imports

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import seaborn as sns
from oggm.core.massbalance import LinearMassBalance as oggm_MassBalance
from oggm.core.flowline import FluxBasedModel as oggm_FluxModel
from oggm.core.flowline import RectangularBedFlowline

from oggm import cfg
cfg.initialize()

# Experiments starting from initial surface height, use rectangular linear constant retreating

## Define colors

In [ ]:
colors = sns.color_palette("colorblind")
colors

In [ ]:
axis_color = list(colors[7]) + [1.]
glacier_color = list(colors[9]) + [.5]
outline_color = [0., 0., 0., 1.]
true_bed_color = list(colors[3]) + [1.]
firts_guess_color = list(colors[4]) + [1.]
combine_color = list(colors[0]) + [1.]

## Import dataset for example and extract data

In [ ]:
folder = 'plot_data/'
filename = 'rec_line_cons_ret_bed_hreg10.nc'

with xr.open_dataset(folder + filename) as ds:
    true_bed_h = ds.total_true_bed_h.data
    widths=ds.true_widths.data
    spinup_sfc_h = ds.spinup_sfc_h.data
    distance = ds.coords['total_distance'].data
    ice_mask = ds.ice_mask.data
    first_guessed_bed_ice = ds.first_guessed_bed_h.data
    guessed_bed_ice = ds.guessed_bed_h.data
    years_forward_run = ds.attrs['years_of_model_run']
    ELA_spinup = ds.attrs['mb_ELA'][0]
    ELA_run = ds.attrs['mb_ELA'][1]
    mb_grad = ds.attrs['mb_grad'][0]
    dataset = ds

In [ ]:
dataset

## forward run with spinup

In [ ]:
def calculate_volume_spinup(bed_h,
                            widths,
                            years_forward_run,
                            ELA_spinup,
                            ELA_run,
                            mb_grad,
                            spinup_sfc_h_ref,
                            time_scale=None,
                            volume_scale=None,
                            evolution_interval=5,
                            show_debug_text=True):
    map_dx = 100.
    
    oggm_fl = RectangularBedFlowline(surface_h=bed_h,
                                     bed_h=bed_h,
                                     widths=(widths /
                                             map_dx),
                                     map_dx=map_dx)

    oggm_mb_model = [oggm_MassBalance(ELA_spinup,
                                      mb_grad),
                     oggm_MassBalance(ELA_run,
                                      mb_grad)]

    start_model = oggm_FluxModel(oggm_fl,
                                 mb_model=oggm_mb_model[0],
                                 y0=0.)
    
    # determine spinup time
    start_model.run_until_equilibrium()
    spinup_years = start_model.yr
    if show_debug_text:
        print('spinup_years = ' + str(spinup_years))
    
    # start again from zero ice
    start_model = oggm_FluxModel(oggm_fl,
                                 mb_model=oggm_mb_model[0],
                                 y0=0.)
    # timesteps for spinup
    yrs_spinup = np.arange(0, spinup_years + 1, evolution_interval)
    # timesteps after spinup
    yrs_run = np.arange(5, years_forward_run + 1, evolution_interval)
    
    if time_scale is None:
        time_scale = years_forward_run / spinup_years
    # total timesteps, scale spinup_years to have the same length as years_forward_run
    spinup_years_offset = spinup_years * time_scale
    yrs_total = np.append(np.arange(0, spinup_years + 1, evolution_interval) * time_scale,
                          np.arange(evolution_interval, years_forward_run + 1, evolution_interval) + spinup_years_offset)
    
    if show_debug_text:
        print('len(yrs_spinup) = ' + str(len(yrs_spinup)) + 
              ', len(yrs_run) = ' + str(len(yrs_run)) +
              ', len(yrs_total) = ' + str(len(yrs_total)))

    # Array to fill with data
    nsteps = len(yrs_spinup) + len(yrs_run)
    volume = np.array([])
    
    if volume_scale is None:
        volume_scale = 1
    # Loop
    for yr in yrs_spinup:
        start_model.run_until(yr)
        volume = np.append(volume, start_model.volume_km3 * volume_scale)

    volume_offset = volume[-1] / volume_scale - volume[-1]

    # safe spinup sfc for comparision
    spinup_sfc_h = start_model.fls[-1].surface_h
    assert np.all(np.isclose(spinup_sfc_h, spinup_sfc_h_ref))
    
    # continue run
    retreat_model = oggm_FluxModel(start_model.fls[-1],
                                   mb_model=oggm_mb_model[1],
                                   y0=0.)
 

    for yr in yrs_run:
        retreat_model.run_until(yr)
        volume = np.append(volume, retreat_model.volume_km3 - volume_offset)

    assert len(yrs_total) == len(volume)

    final_surface_h = retreat_model.fls[-1].surface_h

    return yrs_total, volume, spinup_years_offset, volume_offset, final_surface_h

## forward run start from spinup

In [ ]:
def calculate_volume_starting_with_spinup_sfc_h(bed_h,
                                                widths,
                                                years_forward_run,
                                                ELA_run,
                                                mb_grad,
                                                spinup_sfc_h,
                                                volume_offset=0,
                                                evolution_interval=5):
    map_dx = 100.
    
    oggm_fl = RectangularBedFlowline(surface_h=spinup_sfc_h,
                                     bed_h=bed_h,
                                     widths=(widths /
                                             map_dx),
                                     map_dx=map_dx)

    oggm_mb_model = oggm_MassBalance(ELA_run,
                                     mb_grad)

    retreat_model = oggm_FluxModel(oggm_fl,
                                   mb_model=oggm_mb_model,
                                   y0=0.)
    
    # timesteps for forward run
    yrs_run = np.arange(0, years_forward_run + 1, evolution_interval)

    # Array to fill with data
    volume = np.array([])

    for yr in yrs_run:
        retreat_model.run_until(yr)
        volume = np.append(volume, retreat_model.volume_km3 - volume_offset)

    assert len(yrs_run) == len(volume)

    return yrs_run, volume

## put everything together into one figure

In [ ]:
# with sns.axes_style("darkgrid"):
    
fig = plt.figure(figsize=(18, 10))
ax = fig.subplots() 

fontsize = 22
lw_ax = 2.5

# add 'truth' with spinup
yrs_total, volume, spinup_years_offset, volume_offset, final_surface_h = calculate_volume_spinup(
    bed_h=true_bed_h,
    widths=widths,
    years_forward_run=years_forward_run,
    ELA_spinup=ELA_spinup,
    ELA_run=ELA_run,
    mb_grad=mb_grad,
    spinup_sfc_h_ref=spinup_sfc_h,
    time_scale=40/170,
    volume_scale=0.4/1,
    evolution_interval=5,
    show_debug_text=False)

ax.plot(yrs_total, volume,
        c=true_bed_color,
        lw=lw_ax,
        label='measurement (msr) run',
        zorder=2)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# add different volumes of first few iterations include first guess
bed_h_use = np.array([[np.append(first_guessed_bed_ice, true_bed_h[~ice_mask])]])

iterations = [0]    
for iteration in iterations:
    bed_h_use = np.append(bed_h_use,
                          np.append(guessed_bed_ice[iteration],
                                    true_bed_h[~ice_mask])).reshape(-1, len(true_bed_h))

labels = ['first guess run', '1. COMBINE Iteration run']
colors_use = [firts_guess_color, combine_color]

# calculate volume and add to plot
for i, bed_h in enumerate(bed_h_use):
    yrs_run, volume = calculate_volume_starting_with_spinup_sfc_h(bed_h=bed_h,
                                                                  widths=widths,
                                                                  years_forward_run=years_forward_run,
                                                                  ELA_run=ELA_run,
                                                                  mb_grad=mb_grad,
                                                                  spinup_sfc_h=spinup_sfc_h,
                                                                  volume_offset=volume_offset,
                                                                  evolution_interval=5)

    ax.plot(yrs_run + spinup_years_offset, volume,
            c=colors_use[i],
            lw=lw_ax,
            label=labels[i],
            zorder=1)

# add z axis
plt.annotate(text='',
             xy=(0, 0), 
             xytext=(0, 0.042),
             arrowprops=dict(arrowstyle='<-',
                             mutation_scale=25,
                             color=axis_color,
                             lw=1),
             zorder=1
             )
plt.text(0 ,0.043,'V',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=axis_color)

# add x axis
plt.annotate(text='',
             xy=(0, 0), 
             xytext=(85, 0),
             arrowprops=dict(arrowstyle='<-',
                             mutation_scale=25,
                             color=axis_color,
                             lw=1),
             zorder=1
             )
plt.text(85.5 ,0,'t',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=axis_color)

# vertical line for t end
#ax.axvline(40 + spinup_years_offset, ls='--', c=axis_color, lw=2)
ax.plot([40 + spinup_years_offset, 40 + spinup_years_offset],
        [0, 0.03],
        ls='--', c=axis_color, lw=2)
ax.text(40 + spinup_years_offset, -0.0015, r'$t^{e}$',
        c=axis_color,
        fontsize=fontsize,
        verticalalignment='center',
        horizontalalignment='center')

# vertical line for t spinup
ax.axvline(spinup_years_offset, ls='--', c=axis_color, lw=2)
ax.text(spinup_years_offset, -0.0015, r'$t^{s}$',
        c=axis_color,
        fontsize=fontsize,
        verticalalignment='center',
        horizontalalignment='center')

# add (a)
ax.text(-3, 0.044,
        '(a)',
        fontsize=fontsize,
        verticalalignment='center',
        horizontalalignment='center')

ax.legend(fontsize=fontsize, loc='upper right')
ax.set_ylim([0, 0.042])
ax.set_xlim([0, 86])
ax.axis('off')

# start of plot with snapshot at t^spinup
ax2 = fig.subplots()
ax2.set_position([0.3, 0.15, 0.32, 0.4])

# only plot a part of the bed
index = np.arange(62,82)
linewidth_ax2 = 2.5

# plot true bed_h
ax2.plot(distance[index],
         true_bed_h[index],
         c=true_bed_color,
         zorder=3,
         lw=linewidth_ax2)

# plot first guess bed_h
ax2.plot(distance[index],
         np.append(first_guessed_bed_ice, true_bed_h[~ice_mask])[index],
         c=firts_guess_color,
         zorder=2,
         lw=linewidth_ax2)

# plot COMBINE iteration bed_h
ax2.plot(distance[index],
         np.append(guessed_bed_ice[iterations[0]], true_bed_h[~ice_mask])[index],
         c=combine_color,
         zorder=2,
         lw=linewidth_ax2)

# plot surface height
#ax2.plot(distance[index],
#         spinup_sfc_h[index])
# glacier polygon
x_use=distance[index]
x_polygon = np.concatenate((x_use, x_use[::-1]))
y_polygon = np.concatenate((np.append(first_guessed_bed_ice, true_bed_h[~ice_mask])[index], 
                            spinup_sfc_h[index][::-1]))
coord_polygon = np.concatenate((np.expand_dims(x_polygon, axis=1),np.expand_dims(y_polygon, axis=1)), axis=1)
h1 = ax2.add_patch(Polygon(coord_polygon,
                      fc =glacier_color,  
                      ec =outline_color,
                      closed=False,
                      lw = 0.8,
                      zorder=1,
                      label=r'$s_{o}^{s}$'))

#plot final surface height
h2 = ax2.plot(distance[index], final_surface_h[index],
         '--',
         c='black',
         lw=1,
         label=r'$s_{o}^{e}$ msr run')

# add z axis
plt.annotate(text='',
             xy=(6.3, 2050), 
             xytext=(6.3, 2180),
             arrowprops=dict(arrowstyle='<-',
                             mutation_scale=25,
                             color=axis_color,
                             lw=1),
             zorder=1
             )
plt.text(6.3 ,2200,'z',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=axis_color)

# add x axis
plt.annotate(text='',
             xy=(6.3, 2050), 
             xytext=(6.6, 2050),
             arrowprops=dict(arrowstyle='<-',
                             mutation_scale=25,
                             color=axis_color,
                             lw=1),
             zorder=1
             )
plt.text(6.65 ,2050,'x',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=axis_color)

ax2.text(6.25, 1980, r'(b) glacier terminus',
         fontsize=fontsize,
         verticalalignment='center',
         horizontalalignment='left')

ax2.legend([h1, h2[0]],[h1.get_label(), h2[0].get_label()],
           fontsize=fontsize)
ax2.set_xticks([])
ax2.set_yticks([])

fig.savefig('timeevolution_of_experimetns_starting_with_ice.pdf',format='pdf',bbox_inches='tight',dpi=300);

# Experiments starting from ice free topography

## Define colors

In [ ]:
colors = sns.color_palette("colorblind")
colors

In [ ]:
axis_color = list(colors[7]) + [1.]
glacier_color = list(colors[9]) + [.5]
true_bed_color = list(colors[3]) + [1.]
t_spinup_1_color = list(colors[0]) + [1.]
t_spinup_2_color = list(colors[4]) + [1.]

## Import data

In [ ]:
folder = '/home/patrick/UNI/Master/cluster/rectangular_experiments/wide_top_reg_par_experiments/results/'
filename = 'rec_line_cons_ret_spinup_bed_h_150reg10.nc'

with xr.open_dataset(folder + filename) as ds:
    widths = ds.true_widths.data
    distance = ds.coords['total_distance'].data
    ice_mask = ds.ice_mask.data
    
    true_bed_h_170 = ds.total_true_bed_h.data
    ELA_spinup_true_170 = ds.attrs['mb_ELA'][0]
    
    first_guessed_bed_ice_150 = ds.first_guessed_bed_h.data
    ELA_first_guess_150 = ds.first_guess_spinup_ELA.data
    # guessed_bed_ice = ds.guessed_bed_h.data
    years_forward_run = ds.attrs['years_of_model_run']
    
    ELA_run = ds.attrs['mb_ELA'][1]
    mb_grad = ds.attrs['mb_grad'][0]
    dataset = ds

filename = 'rec_line_cons_ret_spinup_bed_h_100reg10.nc'

with xr.open_dataset(folder + filename) as ds:
    first_guessed_bed_ice_100 = ds.first_guessed_bed_h.data
    ELA_first_guess_100 = ds.first_guess_spinup_ELA.data

In [ ]:
dataset

## define forward run

In [ ]:
def calculate_volume_spinup_2(bed_h,
                              widths,
                              years_forward_run,
                              years_spinup,
                              ELA_spinup,
                              ELA_run,
                              mb_grad,
                              time_scale=None,
                              volume_scale=None,
                              evolution_interval=5,
                              show_debug_text=True):
    map_dx = 100.
    
    oggm_fl = RectangularBedFlowline(surface_h=bed_h,
                                     bed_h=bed_h,
                                     widths=(widths /
                                             map_dx),
                                     map_dx=map_dx)

    oggm_mb_model = [oggm_MassBalance(ELA_spinup,
                                      mb_grad),
                     oggm_MassBalance(ELA_run,
                                      mb_grad)]

    start_model = oggm_FluxModel(oggm_fl,
                                 mb_model=oggm_mb_model[0],
                                 y0=0.)
    
    # determine spinup time
    #start_model.run_until_equilibrium()
    #spinup_years = start_model.yr
    #if show_debug_text:
    #    print('spinup_years = ' + str(spinup_years))
    
    # start again from zero ice
    start_model = oggm_FluxModel(oggm_fl,
                                 mb_model=oggm_mb_model[0],
                                 y0=0.)
    # timesteps for spinup
    yrs_spinup = np.arange(0, years_spinup + 1, evolution_interval)
    # timesteps after spinup
    yrs_run = np.arange(5, years_forward_run + 1, evolution_interval)
    
    if time_scale is None:
        time_scale = 1
    # total timesteps, scale spinup_years to have the same length as years_forward_run
    spinup_years_offset = years_spinup * time_scale
    yrs_total = np.append(np.arange(0, years_spinup + 1, evolution_interval) * time_scale,
                          np.arange(evolution_interval, years_forward_run + 1, evolution_interval) + spinup_years_offset)
    
    if show_debug_text:
        print('len(yrs_spinup) = ' + str(len(yrs_spinup)) + 
              ', len(yrs_run) = ' + str(len(yrs_run)) +
              ', len(yrs_total) = ' + str(len(yrs_total)))

    # Array to fill with data
    nsteps = len(yrs_spinup) + len(yrs_run)
    volume = np.array([])
    
    if volume_scale is None:
        volume_scale = 1
    # Loop
    for yr in yrs_spinup:
        start_model.run_until(yr)
        volume = np.append(volume, start_model.volume_km3 * volume_scale)

    volume_offset = volume[-1] / volume_scale - volume[-1]

    # safe spinup sfc for comparision
    # spinup_sfc_h = start_model.fls[-1].surface_h
    # assert np.all(np.isclose(spinup_sfc_h, spinup_sfc_h_ref))
    
    # continue run
    retreat_model = oggm_FluxModel(start_model.fls[-1],
                                   mb_model=oggm_mb_model[1],
                                   y0=0.)
 

    for yr in yrs_run:
        retreat_model.run_until(yr)
        volume = np.append(volume, retreat_model.volume_km3 - volume_offset)

    assert len(yrs_total) == len(volume)

    final_surface_h = retreat_model.fls[-1].surface_h

    return yrs_total, volume, spinup_years_offset, volume_offset, final_surface_h

## put everything together into one figure

In [ ]:
fig = plt.figure(figsize=(18, 7))
ax = fig.subplots() 

fontsize = 22
lw_ax = 2.5
lw = 2

bed_hts = [true_bed_h_170,
           np.append(first_guessed_bed_ice_150,
                     true_bed_h_170[~ice_mask]),
           np.append(first_guessed_bed_ice_150,
                     true_bed_h_170[~ice_mask]),
           np.append(first_guessed_bed_ice_100,
                     true_bed_h_170[~ice_mask]),
           np.append(first_guessed_bed_ice_100,
                     true_bed_h_170[~ice_mask])]
ELAs = [ELA_spinup_true_170,
        ELA_first_guess_150,
        ELA_first_guess_150-150,
        ELA_first_guess_150,
        ELA_first_guess_150-150]
spinup_years = [170, 150, 150, 100, 100]
labels = ['measurement run',
          r'$t^{spinup}_1$, $ELA^{spinup}_1$',
          r'$t^{spinup}_1$, $ELA^{spinup}_2$',
          r'$t^{spinup}_2$, $ELA^{spinup}_1$',
          r'$t^{spinup}_2$, $ELA^{spinup}_2$']
colors_for_runs = [true_bed_color,
                   t_spinup_1_color,
                   t_spinup_1_color,
                   t_spinup_2_color,
                   t_spinup_2_color]
line_styles = ['-',
               '-.',
               '--',
               '-.',
               '--']
    
for ELA_spinup, spinup_yrs, label, color, linestyle in zip(ELAs,
                                                          spinup_years,
                                                          labels,
                                                          colors_for_runs,
                                                          line_styles):
    yrs_total, volume, spinup_years_offset, volume_offset, final_surface_h = calculate_volume_spinup_2(
        bed_h=bed_hts[0],
        widths=widths,
        years_forward_run=years_forward_run,
        years_spinup=spinup_yrs,
        ELA_spinup=ELA_spinup,
        ELA_run=ELA_run,
        mb_grad=mb_grad,
        time_scale=None,
        volume_scale=None,
        evolution_interval=5,
        show_debug_text=False)

    ax.plot(yrs_total + 170 - spinup_yrs,
            volume,
            linestyle,
            label=label,
            c=color,
            lw=lw)

# add z axis
plt.annotate(text='',
             xy=(0, 0), 
             xytext=(0, 0.1),
             arrowprops=dict(arrowstyle='<-',
                             mutation_scale=25,
                             color=axis_color,
                             lw=lw_ax),
             zorder=1
             )
plt.text(0 ,0.103,'V',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=axis_color)

# add x axis
plt.annotate(text='',
             xy=(0, 0), 
             xytext=(225, 0),
             arrowprops=dict(arrowstyle='<-',
                             mutation_scale=25,
                             color=axis_color,
                             lw=lw_ax),
             zorder=1
             )
plt.text(225.5 ,0,'t',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=axis_color)

# annotation for t end and tspinup
t_y_offset = -0.005
# vertical line for t end
ax.axvline(210, ls='--', c=axis_color, lw=lw)
#ax.plot([40 + spinup_years_offset, 40 + spinup_years_offset],
#        [0, 0.03],
#        ls='--', c=axis_color, lw=2)
ax.text(210, t_y_offset, r'$t^{e}$',
        c=axis_color,
        fontsize=fontsize,
        verticalalignment='center',
        horizontalalignment='center')

# vertical line for t spinup
ax.axvline(170, ls='--', c=axis_color, lw=lw)
ax.text(170, t_y_offset, r'$t^{spinup}/t^{s}$',
        c=axis_color,
        fontsize=fontsize,
        verticalalignment='center',
        horizontalalignment='center')

# create legend
h1 = ax.plot([], [],
             '-',
             c=true_bed_color,
             lw=lw,
             label='measurement run')
h2 = ax.plot([], [],
             '-.',
             c='black',
             lw=lw,
             label=r'ELA^{spinup}_1')
h3 = ax.plot([], [],
             '--',
             c='black',
             lw=lw,
             label=r'ELA^{spinup}_2')
h4 = ax.plot([], [],
             '-',
             c=t_spinup_1_color,
             lw=lw,
             label=r'$t^{spinup}_1$')
h5 = ax.plot([], [],
             '-',
             c=t_spinup_2_color,
             lw=lw,
             label=r'$t^{spinup}_2$')

ax.legend((h1[0], h2[0], h3[0], h4[0], h5[0]),
          ('msr run',
           r'$ELA^{spinup}_1$',
           r'$ELA^{spinup}_2$',
           r'$t^{spinup}_1$',
           r'$t^{spinup}_2$'),
          fontsize=fontsize)

ax.set_xlim([0, 230])
ax.set_ylim([0,0.11])
ax.axis('off')
fig.savefig('timeevolution_of_experimetns_starting_from_ice_free_topography.pdf',format='pdf',bbox_inches='tight',dpi=300);